# NER BERTimbau

In [2]:
! pip install datasets transformers seqeval

# Imports iniciais

In [1]:
task = "ner"
model_checkpoint = "pierreguillou/bert-base-cased-pt-lenerbr"

In [38]:
import random
import pandas as pd
from datasets import load_dataset, load_metric
from datasets import ClassLabel, Sequence
from IPython.display import display, HTML
from transformers import AutoTokenizer
import transformers
from transformers import DataCollatorForTokenClassification

In [ ]:
PATH_DATASET_NER = "../datasets/NER_dataset.json"

In [ ]:
df = pd.read_json(PATH_DATASET_NER)
df.head()

,annotations
0,"[[De, jeito, nenhum.], [ENTITY_PREFERENCE, ENT..."
1,"[[Super-herói, de, ficção, científica], [ENTIT..."
2,"[[Eu, gosto], [ENTITY_PREFERENCE, ENTITY_PREFE..."
3,"[[Bem,, como, eu, disse, eu, eu, gosto, princi..."
4,"[[Sim,, absolutamente.], [ENTITY_PREFERENCE, E..."


In [ ]:
ner_df = df["annotations"]

In [6]:
datasets = load_dataset("xtreme", "PAN-X.pt")

Generating test split: 100%|██████████| 10000/10000 [00:00<00:00, 1427459.42 examples/s]


In [7]:
datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
})

In [8]:
datasets["train"][0]

{'tokens': ['Walt', 'Disney', 'World', 'Resort'],
 'ner_tags': [3, 4, 4, 4],
 'langs': ['pt', 'pt', 'pt', 'pt']}

In [9]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)

In [10]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [11]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [12]:
show_random_elements(datasets["train"])

,tokens,ner_tags,langs
0,"[', '', Radek, Štěpánek, '', '6–4, ,, 6–2]","[O, O, B-PER, I-PER, O, O, O, O]","[pt, pt, pt, pt, pt, pt, pt, pt]"
1,"[Santa, Cristina, de, la, Polvorosa]","[B-LOC, I-LOC, I-LOC, I-LOC, I-LOC]","[pt, pt, pt, pt, pt]"
2,"[REDIRECIONAMENTO, Jaap, ten, Kortenaar]","[O, B-PER, I-PER, I-PER]","[pt, pt, pt, pt]"
3,"[Dario, Cataldo, sofreu, uma, forte, queda, na, 19ª, etapa, .]","[B-PER, I-PER, O, O, O, O, O, O, O, O]","[pt, pt, pt, pt, pt, pt, pt, pt, pt, pt]"
4,"[Marin, Čilić, Andy, Murray, Stan, Wawrinka, Nick, Kyrgios]","[B-PER, I-PER, B-PER, I-PER, B-PER, I-PER, B-PER, I-PER]","[pt, pt, pt, pt, pt, pt, pt, pt]"
5,"[Sei, Ashina, -, A, Garota]","[B-PER, I-PER, O, O, O]","[pt, pt, pt, pt, pt]"
6,"[Eleição, realizada, pela, Assembleia, Legislativa, de, Santa, Catarina, .]","[O, O, O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O]","[pt, pt, pt, pt, pt, pt, pt, pt, pt]"
7,"[poly, 529, 328, 514, 338, 509, 352, 513, 393, 490, 409, 475, 430, 476, 449, 492, 445, 520, 460, 526, 490, 519, 510, 532, 517, 545, 525, 566, 469, 600, 425, 597, 408, 560, 390, 551, 382, 556, 345, Maurice, de, Broglie]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PER, I-PER, I-PER]","[pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt, pt]"
8,"[San, Vicente, del, Raspeig]","[B-LOC, I-LOC, I-LOC, I-LOC]","[pt, pt, pt, pt]"
9,"[REDIRECIONAMENTO, Saint-François-de-Sales, (, Saboia, )]","[O, B-LOC, I-LOC, I-LOC, I-LOC]","[pt, pt, pt, pt, pt]"


# Preprocessamento

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [21]:
label_all_tokens = True

In [22]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, max_length=512)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [23]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[101, 13736, 5390, 4034, 4151, 2532, 102], [101, 112, 112, 112, 5816, 119, 7016, 19191, 112, 112, 112, 102], [101, 1094, 1127, 125, 16347, 120, 4950, 293, 4844, 120, 4709, 4602, 252, 102], [101, 4753, 17035, 1379, 1071, 431, 1123, 12717, 22305, 3073, 102], [101, 112, 112, 112, 8315, 580, 385, 11975, 17675, 22281, 112, 112, 112, 678, 1379, 888, 117, 888, 1379, 245, 117, 888, 1379, 245, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 3, 4, 4, 4, 4, -100], [-100, 0, 0, 0, 3, 3, 4, 4, 0, 0, 0, -100], [-100, 5, 5, 6, 6, 0, 5, 5, 6, 0

In [24]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map: 100%|██████████| 10000/10000 [00:00<00:00, 27599.17 examples/s]


# Fine-tuning

In [25]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at pierreguillou/bert-base-cased-pt-lenerbr and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
model_name = model_checkpoint.split("/")[-1]


per_device_batch_size = 4
gradient_accumulation_steps = 2

learning_rate = 2e-5 
num_train_epochs = 3 
weight_decay = 0.01

save_total_limit = 3
logging_steps = 290 
eval_steps = logging_steps
evaluation_strategy = 'steps'
logging_strategy = 'steps'
save_strategy = 'steps'
save_steps = logging_steps
load_best_model_at_end = True


model_name = model_checkpoint.split("/")[-1]
folder_model = 'e' + str(num_train_epochs) + '_lr' + str(learning_rate)
output_dir = '/content/drive/MyDrive/' + 'ner-lenerbr-' + str(model_name) + '/checkpoints/' + folder_model
logging_dir = '/content/drive/MyDrive/' + 'ner-lenerbr-' + str(model_name) + '/logs/' + folder_model

metric_for_best_model = 'eval_f1'
if metric_for_best_model == 'eval_f1':
    greater_is_better = True
elif metric_for_best_model == 'eval_loss':
    greater_is_better = False  

args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=learning_rate,
    per_device_train_batch_size=per_device_batch_size,
    per_device_eval_batch_size=per_device_batch_size*2,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    save_total_limit=save_total_limit,
    logging_steps = logging_steps,
    eval_steps = logging_steps,
    load_best_model_at_end = load_best_model_at_end,
    metric_for_best_model = metric_for_best_model,
    greater_is_better = greater_is_better,
    gradient_checkpointing = False,
    do_train = True,
    do_eval = True,
    do_predict = True,
    evaluation_strategy = evaluation_strategy,
    logging_dir=logging_dir, 
    logging_strategy = logging_strategy,
    save_strategy = save_strategy,
    save_steps = save_steps,
    push_to_hub=False,
)

In [29]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [30]:
metric = load_metric("seqeval")

C:\Users\bianc\AppData\Local\Temp\ipykernel_24496\152412463.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
c:\Users\bianc\Desktop\Chatbot-NLP\.venv\lib\site-packages\datasets\load.py:753: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [31]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [32]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [33]:
from transformers.trainer_callback import EarlyStoppingCallback

# wait early_stopping_patience x eval_steps before to stp the training in order to get a better model
early_stopping_patience = save_total_limit

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)],
)

In [34]:
trainer.train()

  4%|▍         | 290/7500 [04:44<1:54:23,  1.05it/s]

{'loss': 0.6643, 'grad_norm': 6.410598278045654, 'learning_rate': 1.922666666666667e-05, 'epoch': 0.12}


                                                    
  4%|▍         | 290/7500 [09:13<1:54:23,  1.05it/s]

{'eval_loss': 0.3577207624912262, 'eval_precision': 0.7694964627548897, 'eval_recall': 0.7947990543735225, 'eval_f1': 0.7819431229516863, 'eval_accuracy': 0.8964331797235023, 'eval_runtime': 269.3296, 'eval_samples_per_second': 37.129, 'eval_steps_per_second': 4.641, 'epoch': 0.12}


  8%|▊         | 580/7500 [13:59<1:46:20,  1.08it/s]  

...

                                                     
 93%|█████████▎| 6960/7500 [3:41:25<08:34,  1.05it/s]

{'eval_loss': 0.24479155242443085, 'eval_precision': 0.9045975542204611, 'eval_recall': 0.9125295508274232, 'eval_f1': 0.9085462404245302, 'eval_accuracy': 0.9510875576036867, 'eval_runtime': 267.9869, 'eval_samples_per_second': 37.315, 'eval_steps_per_second': 4.664, 'epoch': 2.78}


 93%|█████████▎| 6960/7500 [3:41:26<17:10,  1.91s/it]

{'train_runtime': 13286.5037, 'train_samples_per_second': 4.516, 'train_steps_per_second': 0.564, 'train_loss': 0.19588431106216606, 'epoch': 2.78}


TrainOutput(global_step=6960, training_loss=0.19588431106216606, metrics={'train_runtime': 13286.5037, 'train_samples_per_second': 4.516, 'train_steps_per_second': 0.564, 'train_loss': 0.19588431106216606, 'epoch': 2.78})

In [35]:
trainer.evaluate()

100%|██████████| 1250/1250 [04:25<00:00,  4.70it/s]


{'eval_loss': 0.2409074753522873,
 'eval_precision': 0.9089310226492794,
 'eval_recall': 0.9107672469374597,
 'eval_f1': 0.9098482083431737,
 'eval_accuracy': 0.9519723502304147,
 'eval_runtime': 266.1832,
 'eval_samples_per_second': 37.568,
 'eval_steps_per_second': 4.696,
 'epoch': 2.78}

In [36]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

100%|██████████| 1250/1250 [04:25<00:00,  4.70it/s]


{'LOC': {'precision': 0.9305513557101098,
  'recall': 0.9342238413276664,
  'f1': 0.9323839822249155,
  'number': 9882},
 'ORG': {'precision': 0.8586437440305635,
  'recall': 0.8770731707317073,
  'f1': 0.8677606177606177,
  'number': 6150},
 'PER': {'precision': 0.9231959487972992,
  'recall': 0.90736900317987,
  'f1': 0.9152140566169293,
  'number': 7233},
 'overall_precision': 0.9089310226492794,
 'overall_recall': 0.9107672469374597,
 'overall_f1': 0.9098482083431737,
 'overall_accuracy': 0.9519723502304147}

In [37]:
model_dir = 'ner_transformers/'
trainer.save_model(model_dir)